In [2]:
import os
import cv2
import numpy as np
import time
from sklearn.preprocessing import OneHotEncoder
import random
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix


In [9]:
class SingleLayerNN:
    def __init__(self, input_size, output_size, learning_rate=0.01):
        self.weights = np.random.randn(input_size, output_size) * 0.01  # Small random values
        self.bias = np.zeros((1, output_size))
        self.learning_rate = learning_rate

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)  # Derivative of sigmoid

    def forward(self, X):
        self.input = X
        self.z = np.dot(X, self.weights) + self.bias  # Linear transformation
        self.output = self.sigmoid(self.z)  # Apply activation
        return self.output

    def backward(self, y_true):
        error = self.output - y_true  # Error in prediction
        d_output = error * self.sigmoid_derivative(self.output)  # Delta (Gradient)
        
        d_weights = np.dot(self.input.T, d_output)  # Weight gradient
        d_bias = np.sum(d_output, axis=0, keepdims=True)  # Bias gradient

        # Update weights and bias using gradient descent
        self.weights -= self.learning_rate * d_weights
        self.bias -= self.learning_rate * d_bias

        loss = np.mean(error**2)  # Mean Squared Error
        return loss

    def train(self, X, y, epochs=100, X_test=None, y_test=None):
        start_time = time.time()
        
        for epoch in range(epochs):
            output = self.forward(X)
            loss = self.backward(y)
            
            if epoch % 10 == 0:
                print(f"Epoch {epoch}, Loss: {loss:.4f}")
        
        end_time = time.time()
        print(f"Total Training Time: {end_time - start_time:.2f} seconds")
        
        if X_test is not None and y_test is not None:
            self.evaluate(X_test, y_test)

    def predict(self, X):
        return (self.forward(X) > 0.5).astype(int)  # Convert probabilities to class labels
    
    def evaluate(self, X_test, y_test):
        y_pred = self.predict(X_test)
        
        accuracy = accuracy_score(y_test, np.argmax(y_pred, axis=1))
        f1 = f1_score(y_test, np.argmax(y_pred, axis=1), average='macro')
        conf_matrix = confusion_matrix(y_test, np.argmax(y_pred, axis=1))
        
        print(f"Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}")
        print("Confusion Matrix:\n", conf_matrix)



In [7]:

# Paths to dataset
train_path = "/kaggle/input/cifar10-pngs-in-folders/cifar10/train"
test_path = "/kaggle/input/cifar10-pngs-in-folders/cifar10/test"

# Function to load CIFAR-10 images and labels
def load_cifar10(dataset_path):
    categories = sorted(os.listdir(dataset_path))  # Sort categories for consistency
    label_map = {category: idx for idx, category in enumerate(categories)}

    images, labels = [], []
    for category in categories:
        category_path = os.path.join(dataset_path, category)
        for img_file in os.listdir(category_path):
            img_path = os.path.join(category_path, img_file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Convert to grayscale
            img = cv2.resize(img, (32, 32))  # Ensure 32x32 size
            images.append(img.flatten() / 255.0)  # Normalize and flatten
            labels.append(label_map[category])

    return np.array(images), np.array(labels)

# Load dataset
X_train, y_train = load_cifar10(train_path)
X_test, y_test = load_cifar10(test_path)

# One-hot encode labels
one_hot_encoder = OneHotEncoder(sparse=False)
y_train_onehot = one_hot_encoder.fit_transform(y_train.reshape(-1, 1))
y_test_onehot = one_hot_encoder.transform(y_test.reshape(-1, 1))


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [10]:

# Initialize and train model
input_size = 32 * 32  # Image size after flattening
output_size = 10  # Number of classes

nn = SingleLayerNN(input_size, output_size, learning_rate=0.1)
nn.train(X_train, y_train_onehot, epochs=100)

# Evaluate model
nn.evaluate(X_test, y_test)

Epoch 0, Loss: 0.2586
Epoch 10, Loss: 0.1000
Epoch 20, Loss: 0.1000
Epoch 30, Loss: 0.1000
Epoch 40, Loss: 0.1000
Epoch 50, Loss: 0.1000
Epoch 60, Loss: 0.1000
Epoch 70, Loss: 0.1000
Epoch 80, Loss: 0.1000
Epoch 90, Loss: 0.1000
Total Training Time: 19.72 seconds
Accuracy: 0.1000, F1-score: 0.0182
Confusion Matrix:
 [[1000    0    0    0    0    0    0    0    0    0]
 [1000    0    0    0    0    0    0    0    0    0]
 [1000    0    0    0    0    0    0    0    0    0]
 [1000    0    0    0    0    0    0    0    0    0]
 [1000    0    0    0    0    0    0    0    0    0]
 [1000    0    0    0    0    0    0    0    0    0]
 [1000    0    0    0    0    0    0    0    0    0]
 [1000    0    0    0    0    0    0    0    0    0]
 [1000    0    0    0    0    0    0    0    0    0]
 [1000    0    0    0    0    0    0    0    0    0]]


In [11]:
class Multilayer_Perceptron(object):
    def __init__(self, sizes):
        self.layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y,1) for y in sizes[1:]]
        self.weights = [np.random.randn(y,x) for x, y in zip(sizes[:-1], sizes[1:])]


    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = self.sigmoid(np.dot(w,a)+b)
        return a

    def backpropagation(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        activation = x
        activation_list = [x]
        z_list = []
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            z_list.append(z)
            activation = self.sigmoid(z)
            activation_list.append(activation)

        delta = self.cost_derivative(activation_list[-1], y) * self.sigmoid_prime(z_list[-1]) 
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activation_list[-2].transpose())

        for layer in range(2, self.layers):
            z = z_list[-layer]
            sp = self.sigmoid_prime(z)
            delta = np.dot(self.weights[-layer+1].transpose(), delta)*sp
            nabla_b[-layer] = delta
            nabla_w[-layer] = np.dot(delta, activation_list[-layer-1].transpose())

        return (nabla_b, nabla_w)


    def update_mini_batch(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backpropagation(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]

        self.weights = [w-(eta/len(mini_batch))*nw for w,nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]


    def Stochastic_Gradient_Descent(self, epochs, training_data, mini_batch_size, eta, test_data=None):
        n= len(training_data)
        
        if test_data:
            test_n = len(test_data)
            #n = len(training_data)
        start_time = time.time()
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)

            if test_data:
                #print('here')
                #print("Epoch{0}: {1}/{2}".format(j, self.evaluate(test_data), test_n))
                accuracy, f1, conf_matrix = self.evaluate(test_data)
                print(f"Epoch {j}: Accuracy = {accuracy:.4f}, F1-score = {f1:.4f}")
                print("Confusion Matrix:\n", conf_matrix)
       
            else:
                #print('code is here')
                print("Epoch{0} complete".format(j))
        end_time = time.time()
        total_time = end_time-start_time
        print(f"Total Training Time: {end_time - start_time:.2f} seconds")

    def cost_derivative(self, output_activations, y):
        return (output_activations-y)
    
    def sigmoid(self, z):
        return 1/(1+np.exp(-z))

    def sigmoid_prime(self, z):
        result = self.sigmoid(z)*(1-self.sigmoid(z))
        return result


    def evaluate(self, test_data):
        y_pred = []
        y_true = []
        for x, y in test_data:
            predicted = np.argmax(self.feedforward(x))
            actual = np.argmax(y)
            y_pred.append(predicted)
            y_true.append(actual)
        #test_results = [(np.argmax(self.feedforward(x)), np.argmax(y)) for (x, y) in test_data]
        #return sum(int(x == y) for (x, y) in test_results)
        accuracy = accuracy_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred, average="macro")
        conf_matrix = confusion_matrix(y_true, y_pred)

        return accuracy, f1, conf_matrix


        
        


In [12]:

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize pixel values to [0,1]
x_train = x_train.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0

# Flatten images into vectors
x_train = x_train.reshape(x_train.shape[0], -1, 1)  # (50000, 3072, 1)
x_test = x_test.reshape(x_test.shape[0], -1, 1)    # (10000, 3072, 1)

# One-hot encode labels
def one_hot_encode(y, num_classes=10):
    return np.eye(num_classes)[y].reshape(-1, num_classes, 1)

y_train = one_hot_encode(y_train)
y_test = one_hot_encode(y_test)

# Prepare training and test data
td = list(zip(x_train, y_train))
test_data = list(zip(x_test, y_test))


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 12s 0us/step


In [13]:
mlp = Multilayer_Perceptron([3072, 128, 64, 10])
mlp.Stochastic_Gradient_Descent(epochs=10, training_data=td, mini_batch_size=32, eta=0.01, test_data=test_data)


Epoch 0: Accuracy = 0.1000, F1-score = 0.0182
Confusion Matrix:
 [[   0    0    0 1000    0    0    0    0    0    0]
 [   0    0    0 1000    0    0    0    0    0    0]
 [   0    0    0 1000    0    0    0    0    0    0]
 [   0    0    0 1000    0    0    0    0    0    0]
 [   0    0    0  998    0    1    0    0    0    1]
 [   0    0    0 1000    0    0    0    0    0    0]
 [   0    0    0  999    0    0    0    1    0    0]
 [   0    0    0 1000    0    0    0    0    0    0]
 [   0    0    0 1000    0    0    0    0    0    0]
 [   0    0    0  999    0    0    0    1    0    0]]
Epoch 1: Accuracy = 0.0999, F1-score = 0.0184
Confusion Matrix:
 [[   0    0    0 1000    0    0    0    0    0    0]
 [   0    0    0  998    0    0    0    2    0    0]
 [   0    1    0  999    0    0    0    0    0    0]
 [   0    0    0  998    0    0    0    1    0    1]
 [   0    0    0  998    0    1    0    0    0    1]
 [   0    0    0  998    0    0    0    2    0    0]
 [   0    0    0  998